In [15]:
import numpy as np
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.callbacks import ModelCheckpoint
import cv2
from sklearn.externals import joblib
import pickle

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator

In [3]:
trainList = []
trainLabels = []
testList = []
testLabels = []

In [29]:
folders = ['drawings', 'engraving', 'iconography', 'painting', 'sculpture']

In [5]:
labels = {'drawings': [1,0,0,0,0],
          'engraving': [0,1,0,0,0],
          'iconography': [0,0,1,0,0],
          'painting': [0,0,0,1,0],
          'sculpture': [0,0,0,0,1]}

In [6]:
for folder in folders:
    count = 0
    for filename in glob.glob('art-images/dataset/dataset_updated/training_set/' + folder +'/*'):
        try:
            count = count + 1
            im = Image.open(filename).convert('RGB')
            im = im.resize((64,64))
            imArr = np.array(im)
            trainList.append(imArr)
        
            trainLabels.append(labels[folder])
        except OSError as e:
            pass
    print(folder + ": ")
    print(count)
    #print(np.array(trainList).shape)

drawings: 
1107
engraving: 
757


c:\users\vesko\appdata\local\programs\python\python36\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6029312 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
c:\users\vesko\appdata\local\programs\python\python36\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1311848 bytes but only got 785. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
c:\users\vesko\appdata\local\programs\python\python36\lib\site-packages\PIL\TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


iconography: 
2077
painting: 
2042


c:\users\vesko\appdata\local\programs\python\python36\lib\site-packages\PIL\Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


sculpture: 
1738


In [7]:
for folder in folders:
    count = 0
    for filename in glob.glob('art-images/dataset/dataset_updated/validation_set/' + folder +'/*'):
        try:
            count = count+1
            im = Image.open(filename).convert('RGB')
            im = im.resize((64,64))
            imArr = np.array(im)
            testList.append(imArr)
            testLabels.append(labels[folder])
        except OSError as e:
            pass
    print(folder + ": ")
    print(count)

drawings: 
122
engraving: 
84
iconography: 
231
painting: 
228
sculpture: 
191


In [8]:
np.array(testList).shape

(856, 64, 64, 3)

In [9]:
np.array(testLabels).shape

(856, 5)

In [10]:
np.array(trainList).shape

(7721, 64, 64, 3)

In [11]:
np.array(trainLabels).shape

(7721, 5)

In [12]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu', padding="same",
                 input_shape = (64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))



model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5))
model.add(Activation('softmax'))

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
np.array(trainList).shape

(7721, 64, 64, 3)

In [15]:
trainArr = np.array(trainList)
trainArr = trainArr/255.0


In [16]:
testArr = np.array(testList)
testArr = testArr / 255.0

In [19]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [20]:
filepath="best_weights_of_old.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max', period=10)

In [21]:
model.fit_generator(datagen.flow(trainArr, np.array(trainLabels), batch_size=16),steps_per_epoch=230, epochs=40, callbacks = [checkpoint])

Epoch 1/40
230/230 [==============================] - 1911s 8s/step - loss: 1.4326 - acc: 0.3535
Epoch 2/40
230/230 [==============================] - 365s 2s/step - loss: 1.0313 - acc: 0.6003
Epoch 3/40
230/230 [==============================] - 479s 2s/step - loss: 0.9015 - acc: 0.6610
Epoch 4/40
230/230 [==============================] - 478s 2s/step - loss: 0.8931 - acc: 0.6663
Epoch 5/40
230/230 [==============================] - 283s 1s/step - loss: 0.8152 - acc: 0.6996
Epoch 6/40
230/230 [==============================] - 480s 2s/step - loss: 0.7635 - acc: 0.7149
Epoch 7/40
230/230 [==============================] - 479s 2s/step - loss: 0.7428 - acc: 0.7347
Epoch 8/40
230/230 [==============================] - 415s 2s/step - loss: 0.6998 - acc: 0.7633
Epoch 9/40
230/230 [==============================] - 391s 2s/step - loss: 0.6544 - acc: 0.7654
Epoch 10/40
230/230 [==============================] - 233s 1s/step - loss: 0.6470 - acc: 0.7663
Epoch 11/40


c:\users\vesko\appdata\local\programs\python\python36\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


230/230 [==============================] - 229s 994ms/step - loss: 0.6049 - acc: 0.7838
Epoch 12/40
230/230 [==============================] - 226s 983ms/step - loss: 0.6476 - acc: 0.7753
Epoch 13/40
230/230 [==============================] - 226s 983ms/step - loss: 0.5889 - acc: 0.7923
Epoch 14/40
230/230 [==============================] - 230s 1000ms/step - loss: 0.5966 - acc: 0.7932
Epoch 15/40
230/230 [==============================] - 227s 986ms/step - loss: 0.5548 - acc: 0.8039
Epoch 16/40
230/230 [==============================] - 226s 981ms/step - loss: 0.5191 - acc: 0.8147
Epoch 17/40
230/230 [==============================] - 226s 982ms/step - loss: 0.5265 - acc: 0.8099
Epoch 18/40
230/230 [==============================] - 226s 981ms/step - loss: 0.5032 - acc: 0.8201
Epoch 19/40
230/230 [==============================] - 225s 978ms/step - loss: 0.5101 - acc: 0.8168
Epoch 20/40
230/230 [==============================] - 234s 1s/step - loss: 0.5087 - acc: 0.8095
Epoch 21/40
23

In [22]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [17]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("best_weights_of_old.hdf5")

In [18]:
loaded_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
filepath = 'art-images\\dataset\\dataset_updated\\validation_set\\painting\\0006.jpg'

In [32]:
im = Image.open(filepath).convert('RGB')

In [33]:
im = im.resize((64, 64))
imArr = np.array(im)
x = np.expand_dims(imArr, axis=0)
x = x / 255.0

In [34]:
res = loaded_model.predict_classes(x)

In [35]:
folders[res[0]]

'painting'

In [27]:
score = model.evaluate(testArr, np.array(testLabels)) 

NameError: name 'testArr' is not defined

In [25]:
score

[0.5226591076249274, 0.8130841127065854]

In [21]:
model.metrics_names

['loss', 'acc']